# Get new parametersets with Baysian Optimization

This script uses the BaysianBackEnd (BayBE). For more information you can find the documentation here: https://emdgroup.github.io/baybe/stable/index.html 

In [1]:
%%capture
!pip install baybe
import pandas as pd
import numpy as np
import datetime
import math

from baybe.targets import NumericalTarget
from baybe.objectives.single import SingleTargetObjective
from baybe.constraints import ContinuousLinearInequalityConstraint

import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
# TODO maybe let the user set these things in the future

parameters_data = [
    ("potential1",-2000,1600,1),
    ("potential3",-2000,400,1),
    ("hold1",0,1800,1),
    ("sweep_speed",10,1000,1),
]
#("potential2",900,900,1),
#("hold2",0,0,1),
# ("cycles",100,100,1),

In [3]:
# all ipywidgets
author_selector = widgets.Dropdown(
    options=['Maitryi Gupta', 'Literature-Marlena Thormeier', 'FFT-Marlena Thormeier'],
    value=None,
    description='NOMAD author:',
    style={'description_width': 'initial'}
)

run_baybe_button = widgets.Button(
    description='Run Bayesian Optimization',
    button_style='primary',
    layout=widgets.Layout(width='auto')
)

safe_recommendations_button = widgets.Button(
    description='Safe recommended parameters as csv',
    button_style='primary',
    layout=widgets.Layout(width='auto')
)

author_measurement_overview_output = widgets.Output()
recommendations_output = widgets.Output()
save_csv_output = widgets.Output()

In [4]:
def on_author_change(change):
    global measurements
    if change['type'] == 'change' and change['name'] == 'value':
        first_name = author_selector.value.split()[0].lower()
        file_name = f'baybe_csv/wateroxidation_ni_{first_name}.csv'
        with author_measurement_overview_output:
            author_measurement_overview_output.clear_output()
            try:
                measurements = pd.read_csv(file_name)
                print('Measurements for given user:')
                display(measurements)
            except FileNotFoundError:
                print(f'Could not find file {file_name}')
            

author_selector.observe(on_author_change)
author_selector.value = 'Maitryi Gupta'

display(author_selector, author_measurement_overview_output)

Dropdown(description='NOMAD author:', options=('Maitryi Gupta', 'Marlena Thormeier'), style=DescriptionStyle(d…

Output()

In [5]:
def scale_measurements_down(df, parameter_info):
    for p in parameter_info:
        values = (df[p[0]]-p[1])/(p[2]-p[1])
        df[p[0]] = values
    return df

def scale_parameters_up(df, parameter_info):
    for p in parameter_info:
        values = (p[2]-p[1])*df[p[0]] + p[1]
        df[p[0]] = round(values / p[3]) * p[3]
    return df

In [6]:
from baybe.parameters import NumericalContinuousParameter
from baybe.searchspace import SearchSpace
from baybe.recommenders import (
    BotorchRecommender,
    RandomRecommender,
    TwoPhaseMetaRecommender,
)
from baybe import Campaign


def get_clean_baybe_setup(parameters_data):
    constraints = None
    parameters = [
        NumericalContinuousParameter(
            name=p[0],
             bounds=(0, 1),      
        ) for p in parameters_data
    ]
    recommender = TwoPhaseMetaRecommender(
        initial_recommender=RandomRecommender(),  # farthest point sampling
        recommender=BotorchRecommender(sequential_continuous=True),  # Bayesian model-based optimization
    )
    searchspace = SearchSpace.from_product(parameters,constraints=constraints)

    print('Minimize for target cp_geom_mean.')
    target = NumericalTarget(name="cp_geom_mean", mode="MIN")
    objective = SingleTargetObjective(target)
    columns = [p[0] for p in parameters_data]
    campaign = Campaign(searchspace, objective, recommender)
    return campaign

def get_baybe_recommendations(campaign, parameters_data, measurements, num_recommendations=6):
    scaled_measurements = measurements.copy() # use a copy if user hits recommendation button several times
    scaled_measurements = scale_measurements_down(scaled_measurements, parameters_data)
    # print(scaled_measurements) # uncomment to check if measurements are scaled down as expected
    campaign.add_measurements(scaled_measurements)
    df_next = campaign.recommend(batch_size=num_recommendations)
    # print(df_next) #uncomment to see unscaled recommendations
    df_next = scale_parameters_up(df_next, parameters_data)
    return df_next

def get_complete_recommendations(df_next):
    # this function adds longer column names and parameters that we do not change to make the table as expected by the student
    df_next['potential2'] = 900
    df_next['hold2'] = 0
    interval1 = df_next['hold1'] + np.abs(df_next['potential1'] - df_next['potential2'])/df_next['sweep_speed']
    interval2 = np.abs(df_next['potential2'] - df_next['potential3'])/df_next['sweep_speed'] + df_next['hold2']
    df_next['cycles'] = [min(100, math.floor((3600 - interval1[i]) / interval2[i])) for i in range(len(interval1))]
    df_next['duration_s'] = interval1 + df_next['cycles']*interval2
    df_next['duration_min'] = df_next['duration_s']/60
    df_next['duration_h'] = df_next['duration_s']/3600
    rename_columns = {
        'potential1': 'potential1 (mV vs Hg/HgO)',
        'hold1': 'hold1 (s)',
        'potential2': 'potential2 (mV vs Hg/HgO)',
        'hold2': 'hold2 (s)',
        'potential3': 'potential3 (mV vs Hg/HgO)',
        'sweep_speed': 'sweep speed (mV/s)',
        'cycles': 'cycle (P2-P3)',
        'duration_s': 'duration (s)',
        'duration_h': 'duration (h)',
        'duration_min': 'duration (min)',
    }
    df_next = df_next.rename(columns=rename_columns)
    column_order = ['potential1 (mV vs Hg/HgO)', 'hold1 (s)', 'potential2 (mV vs Hg/HgO)', 'hold2 (s)', 'potential3 (mV vs Hg/HgO)', 'sweep speed (mV/s)', 'cycle (P2-P3)', 'duration (s)',	'duration (h)', 'duration (min)']
    df_next = df_next[column_order]
    return df_next

def save_recommendations_as_csv(df_next, author_name):
    date_now = datetime.datetime.now()
    filename = f'baybe_csv/newparameters_ni_{author_name}_{date_now.strftime("%Y%m%d")}.csv'
    df_next.to_csv(filename, index=False, header=True)
    print(f'File saved in {filename}')

In [7]:
def on_baybe_button_clicked(b):
    global df_pretty
    with recommendations_output:
        recommendations_output.clear_output()
        campaign = get_clean_baybe_setup(parameters_data)
        df_next = get_baybe_recommendations(campaign, parameters_data, measurements, num_recommendations=6)
        df_pretty = get_complete_recommendations(df_next)
        print('Recommended next parameter sets:')
        display(df_pretty)

run_baybe_button.on_click(on_baybe_button_clicked)

display(run_baybe_button, recommendations_output)

Button(button_style='primary', description='Run Bayesian Optimization', layout=Layout(width='auto'), style=But…

Output()

In [8]:
def on_save_csv_button_clicked(b):
    with save_csv_output:
        save_csv_output.clear_output()
        first_name = author_selector.value.split()[0].lower()
        save_recommendations_as_csv(df_pretty, first_name)

safe_recommendations_button.on_click(on_save_csv_button_clicked)

display(safe_recommendations_button, save_csv_output)

Button(button_style='primary', description='Safe recommended parameters as csv', layout=Layout(width='auto'), …

Output()